In [1]:
import pandas as pd
import pickle

In [2]:
data_df = pd.read_csv('wiki_intro_processed.csv')

In [18]:
def create_prompt(title, starter_text):
    return f'''200 word wikipedia style introduction on '{title}'
    {starter_text}'''

In [4]:
results = []

for i in range(150):
    with open(f'data/result-{i}.pkl', 'rb') as file:
        temp = pickle.load(file)
        results += list(temp)

In [5]:
len(results)

150000

In [9]:
processed_results = []
for dct in results:
    for key in dct:
        processed_results.append({
            'title': key, 
            'generated_text': dct[key]['choices'][0]['text'],
            'prompt_tokens' : dct[key]['usage']['prompt_tokens'],
            'completion_tokens' : dct[key]['usage']['completion_tokens'],
        })

In [12]:
processed_results_df = pd.DataFrame(processed_results)

In [16]:
# Create final df
final_df = pd.merge(data_df, processed_results_df, on=['title'])

In [19]:
# Create new columns
final_df['prompt'] = final_df.apply(lambda row: create_prompt(row['title'], row['starter_text']), axis=1)

final_df['generated_text_complete'] = final_df['starter_text'] + final_df['generated_text']

final_df['generated_text_len'] = final_df['generated_text_complete'].apply(lambda x: len(x.split(' ')))

In [38]:
# Rename columns
final_df = final_df.rename(columns={
    'intro': 'wiki_intro', 'intro_len': 'wiki_intro_len',
    'generated_text_complete': 'generated_intro', 'generated_text_len' : 'generated_intro_len'})

In [40]:
# Reorder columns
final_df = final_df[[ 
    'id', 'url', 'title', 'wiki_intro', 'generated_intro', 'title_len',
    'wiki_intro_len', 'generated_intro_len', 'prompt', 'generated_text',
    'prompt_tokens', 'generated_text_tokens']]

In [42]:
# Write csv file
final_df.to_csv('GPT-wiki-intro.csv', index=False)